In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

In [84]:
transform_train = transforms.Compose([
    # transforms.Resize((224,224)),
    # transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [85]:
np.unique(np.array(trainset.targets),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]))

In [86]:
np.unique(np.array(testset.targets),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]))

In [87]:
trainset.class_to_idx

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [88]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        
        self.relu = nn.ReLU(inplace=True)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=3, padding=1, bias=False)
        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        identity = self.shortcut(x)
        out = self.conv1(self.relu(self.bn1(x)))
        out = self.conv2(self.relu(self.bn2(out)))
        out += identity
        return out

# ResNet architecture
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 32
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)  # For grayscale images
        self.bn1 = nn.BatchNorm2d(32)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 128, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[3], stride=2)

        self.relu = nn.ReLU(inplace=True)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.relu(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def Resnet():
    return ResNet(BasicBlock, [3, 6, 4, 3])

In [89]:
model = Resnet()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=15, gamma=0.1)

In [90]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
sum([np.prod(p.size()) for p in model_parameters])

4883178

In [91]:
def train(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training hyperparameters
epochs = 50

for epoch in range(1, epochs + 1):
    train(model, device, trainloader, criterion, optimizer, epoch)
    test(model, device, testloader, criterion)
    scheduler.step()


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.410014
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.468668
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.212180
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.359720

Test set: Average loss: 0.0139, Accuracy: 5462/10000 (55%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.038758
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.123184
Train Epoch: 2 [25600/50000 (51%)]	Loss: 0.862034
Train Epoch: 2 [38400/50000 (77%)]	Loss: 0.703086

Test set: Average loss: 0.0085, Accuracy: 7035/10000 (70%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.676484
Train Epoch: 3 [12800/50000 (26%)]	Loss: 0.681581
Train Epoch: 3 [25600/50000 (51%)]	Loss: 0.647969
Train Epoch: 3 [38400/50000 (77%)]	Loss: 0.857744

Test set: Average loss: 0.0076, Accuracy: 7439/10000 (74%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 0.555330
Train Epoch: 4 [12800/50000 (26%)]	Loss: 0.563555
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.401050
Train Epoch: 4 [38400/50000 (77%)]	Loss: 0.357865

Test set: Average loss: 0.0072, Accu

In [11]:
torch.save(model,"model-Resnet30.pt")

In [22]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict1 = pickle.load(fo, encoding='bytes')
    return dict1
def model_test(model, data_path):
    dict = unpickle(data_path)
    data_test = dict[b'data']
    model.eval()
    pred = []

    for i in range(data_test.shape[0]):
        # Reshape from (3072,) to (3, 32, 32) for an RGB image
        img_rgb = data_test[i].reshape(3, 32, 32).transpose((1, 2, 0))

        # Convert to PIL image
        img_pil = Image.fromarray(img_rgb)

        # Apply the transformation
        img_tensor = transform_test(img_pil)

        # Add a batch dimension and send the image to the model
        img_tensor = img_tensor.unsqueeze(0).cuda()
        with torch.no_grad():
            output = model(img_tensor)
            _, predicted = torch.max(output, 1)
            pred.extend(predicted.cpu().numpy())

    return pred

In [92]:
pred = model_test(model, "cifar_test_nolabels.pkl")
np.unique(pred, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 932,  911,  917, 1199,  954, 1058,  926, 1004, 1027, 1072]))

In [30]:
import pandas as pd
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict1 = pickle.load(fo, encoding='bytes')
    return dict1

dict = unpickle("cifar_test_nolabels.pkl")
df = pd.DataFrame({"ID" : dict[b'ids'], "Labels" : pred})

In [27]:
torch.save(model, "Resnet-18-v2-batch256-sub9.pt")

In [31]:
df.to_csv("submission9.csv", index = False)

In [50]:
pd.read_csv("submission10.csv")['Labels'].value_counts()

Labels
3    1131
5    1067
8    1059
9    1056
7    1028
1     961
4     939
6     932
0     925
2     902
Name: count, dtype: int64